In [7]:
import pandas as pd
import numpy as np
from scipy.stats import norm

# Given Parameters
CL = 0.99
z = 2.3263478740408408  # z-score for 99% confidence
total_value = 1_000_000.0
weights = np.array([0.5, 0.3, 0.2])

# Load returns
returns = pd.read_csv("sample_data/3assets_returns.csv", index_col=0, parse_dates=True)

# --- Step 1: compute daily std dev for each asset ---
sigma_assets = returns.std()

# --- Step 2: compute 1-day VaR and ES for each asset ---
# VaR formula (normal): z * sigma * portfolio_value
# ES formula: (pdf(z)/(1-CL)) * sigma * portfolio_value
k = norm.pdf(z) / (1 - CL)

var_1d_assets = z * sigma_assets * total_value
es_1d_assets = k * sigma_assets * total_value

# --- Step 3: scale to 10-day ---
var_10d_assets = var_1d_assets * np.sqrt(10)
es_10d_assets = es_1d_assets * np.sqrt(10)

# --- Step 4: portfolio calculations ---
# covariance matrix
cov_matrix = returns.cov()

# portfolio std dev
sigma_portfolio = np.sqrt(weights @ cov_matrix.values @ weights.T)

# portfolio risk measures
var_1d_port = z * sigma_portfolio * total_value
es_1d_port = k * sigma_portfolio * total_value
var_10d_port = var_1d_port * np.sqrt(10)
es_10d_port = es_1d_port * np.sqrt(10)

# --- Step 5: Collect results into a table ---
results = pd.DataFrame({
    "VaR_1d": np.append(var_1d_assets.values, var_1d_port),
    "ES_1d": np.append(es_1d_assets.values, es_1d_port),
    "VaR_10d": np.append(var_10d_assets.values, var_10d_port),
    "ES_10d": np.append(es_10d_assets.values, es_10d_port)
}, index=list(sigma_assets.index) + ["Portfolio"])

# Show rounded view for easy reading and also keep full precision in CSV
results_rounded = results.round(2)

# Display results
print(results_rounded.round(2))

# Save to CSV
out_path = "sample_data/var_es_historic_results.csv"
results_rounded.to_csv(out_path)


                  VaR_1d     ES_1d    VaR_10d     ES_10d
Asset_A_Return  27581.71  31599.39   87221.04   99926.05
Asset_B_Return  17208.27  19714.91   54417.33   62344.01
Asset_C_Return  45280.72  51876.52  143190.22  164047.95
Portfolio       20187.01  23127.54   63836.93   73135.71
